In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import PIL.Image as pilimg
root_logdir = os.path.join(os.curdir, "my_tb_logs")

In [3]:
emolabel = ['anger', 'fear', 'disgust', 'happiness', 'sadness', 'surprise', 'neutral']
emotype = ['train', 'valid']

In [ ]:
def setimgset(): #Memory Caution
    while True:
        try:
            a, b = str(input("Enter : emotion subset")).split()
            c, d = emolabel.index(a), emotype.index(b)
        except ValueError:
            print("Your input of emotion or subset is either inaccurate or invalid. Try again")
            continue
        else:
            break

    mydir = "KEmoDataset/" + emotype[d] + "_" + emolabel[c] + ".json"
    df = pd.read_json(mydir)
    
    names = df["filename"]
    imlist = []
    i = 0
    while True:
        try:
            datdir = "KEmoDataset/" + emotype[d] + "/" + names[i]
            pix = np.array(pilimg.open(datdir))
        except MemoryError:
            break
        else:
            imlist.append(pix)
            i = i + 1
    return imlist

In [ ]:
def setimgset(): #Memory Over
    while True:
        try:
            a, b = str(input("Enter : emotion subset")).split()
            c, d = emolabel.index(a), emotype.index(b)
        except ValueError:
            print("Your input of emotion or subset is either inaccurate or invalid. Try again")
            continue
        else:
            break

    mydir = "KEmoDataset/" + emotype[d] + "_" + emolabel[c] + ".json"
    df = pd.read_json(mydir)
    
    names = df["filename"]
    imlist = []
    for i in range(len(names)):
        datdir = "KEmoDataset/" + emotype[d] + "/" + names[i]
        pix = np.array(pilimg.open(datdir))
        imlist.append(pix)
    return imlist

In [7]:
def setimgdir():
    while True:
        try:
            a, b = str(input("Enter : emotion subset")).split()
            c, d = emolabel.index(a), emotype.index(b)
        except ValueError:
            print("Your input of emotion or subset is either inaccurate or invalid. Try again")
            continue
        else:
            break

    mydir = "KEmoDataset/" + emotype[d] + "_" + emolabel[c] + ".json"
    df = pd.read_json(mydir)
    
    names = df["filename"]
    imlist = []
    for i in range(len(names)):
        imlist.append(names[i])
    return imlist

In [8]:
input_2 = setimgdir()

Enter : emotion subsetanger train


In [9]:
input_2

['ed8e05a7770502606d8278a279c624e4fb7e681fd09766c71b4c8eb8d3af7a5b_여_30_분노_상업시설&점포&시장_20201206210735-003-003.jpg',
 '6ea444752308b9fc4ce87b2dc63c1341da53b94bc7804a5ea3106732950ca1ac_여_20_분노_도심 환경_20201207140412-008-022.jpg',
 'ca3a4e6e725cbd4765f642703c442507648cefb3eb9262e5254820145386193c_여_20_분노_상업시설&점포&시장_20201209133753-003-011.jpg',
 'aec709ad22845bdbb7103c9e79cede5ac316fd4a3dc0e42f3b4fa7693e397546_여_20_분노_실외 자연환경_20201202131933-007-008.jpg',
 'bc4c9e3eeb74e4d173f26b3a184f41f545c76e7f3a21d2cb9a5751d05ccb8305_여_20_분노_실외 자연환경_20201205220251-007-011.jpg',
 '11c19b42e141ebd621a096717a4d8c31c43104f557201d10878b6f17b8451768_여_20_분노_교통&이동수단(엘리베이터 포함)_20210114181840-002-001.jpg',
 'e728d90328ca632477876500c3fc35b3ca88d690def9d6df2f7f2b51b0db6bf0_여_30_분노_상업시설&점포&시장_20201207130445-003-018.jpg',
 '82ccb7a06bd8182d1581baac54b161a2eef98fdc1626c1a64f1b640d89ece1d9_남_20_분노_도심 환경_20210126000342-008-026.jpg',
 '42f38c12577ba328f23ac4b46572a0ff91e24b00069f6da60ce80058e51a0117_여_20_분노_상업시설&점포&시장_202

In [10]:
def MLP():
    model = keras.models.Sequential()
    model.add(layers.Flatten(input_shape=[28, 28]))
    model.add(layers.Dense(300, activation="relu"))
    model.add(layers.Dense(100, activation="relu"))
    model.add(layers.Dense(10, activation="softmax"))
    return model

In [15]:
def compiler(lr=0,model=MLP(), op="sgd"):
    if op=="sgd":
        optimizer = keras.optimizers.SGD(learning_rate=lr) if lr!=0 else keras.optimizers.SGD()
    elif op=="nes":
        optimizer = keras.optimizers.SGD(learning_rate=lr, momentum=0.9, nesterov=True) if lr!=0 else keras.optimizers.SGD(momentum=0.9, nesterov=True)
    elif op=="adg":
        optimizer = keras.optimizers.Adagrad(learning_rate=lr) if lr!=0 else keras.optimizers.Adagrad()
    elif op=="rms":
        optimizer = keras.optimizers.RMSprop(learning_rate=lr) if lr!=0 else keras.optimizers.RMSprop()
    elif op=="adm":
        optimizer = keras.optimizers.Adam(learning_rate=lr) if lr!=0 else keras.optimizers.Adam()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

In [20]:
def tflearn(name="mobilev3"):
    if name=="mobilev1":
        base_model = keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
    elif name=="mobilev2":
        base_model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
    elif name=="mobilev3s":
        base_model = keras.applications.MobileNetV3Small(input_shape=(224, 224, 3), weights="imagenet", include_preprocessing=False, include_top=False)
    elif name=="mobilev3mins":
        base_model = keras.applications.MobileNetV3Small(input_shape=(224, 224, 3), weights="imagenet", minimalistic=True, include_preprocessing=False, include_top=False)
    elif name=="mobilev3min":
        base_model = keras.applications.MobileNetV3Large(input_shape=(224, 224, 3), weights="imagenet", minimalistic=True, include_preprocessing=False, include_top=False)
    else:
        base_model = keras.applications.MobileNetV3Large(input_shape=(224, 224, 3), weights="imagenet", include_preprocessing=False, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    output = keras.layers.Dense(6, activation="softmax")(avg)
    model = keras.Model(inputs=base_model.input, outputs=output)
    return compiler(model=model, op="adm")

In [21]:
import time
def get_run_logdir():
    run_id = time.strftime("run_%Y_%m_%d_%H_%M_%S")
    return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()

In [18]:
def tflearn(name="mobilev3"): #assume train, valid, test data are ready
    if name=="mobilev1":
        base_model = keras.applications.mobilenet.MobileNet(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
    elif name=="mobilev2":
        base_model = keras.applications.mobilenet_v2.MobileNetV2(input_shape=(224, 224, 3), weights="imagenet", include_top=False)
    elif name=="mobilev3s":
        base_model = keras.applications.MobileNetV3Small(input_shape=(224, 224, 3), weights="imagenet", include_preprocessing=False, include_top=False)
    elif name=="mobilev3mins":
        base_model = keras.applications.MobileNetV3Small(input_shape=(224, 224, 3), weights="imagenet", minimalistic=True, include_preprocessing=False, include_top=False)
    elif name=="mobilev3min":
        base_model = keras.applications.MobileNetV3Large(input_shape=(224, 224, 3), weights="imagenet", minimalistic=True, include_preprocessing=False, include_top=False)
    else:
        base_model = keras.applications.MobileNetV3Large(input_shape=(224, 224, 3), weights="imagenet", include_preprocessing=False, include_top=False)
    for layer in base_model.layers:
        layer.trainable = False
    avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
    output = keras.layers.Dense(6, activation="softmax")(avg)
    model = keras.Model(inputs=base_model.input, outputs=output)
    model = compiler(model=model, op="adm")
    model.fit(train_set, epochs=5, validation_data=valid_set)
    
    for layer in base_model.layers:
        layer.trainable = True
    model = compiler(lr=1e-5, model=model, op="adm")
    filename = name + time.strftime("_%Y_%m_%d_%H_%M_%S") + ".h5"
    cp = keras.callbacks.ModelCheckpoint(filename, save_best_only=True)
    es = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    tb = keras.callbacks.TensorBoard(run_logdir)
    history = model.fit(train_set, epochs=100, validation_data=valid_set, callbacks=[cp, es, tb])
    return model.evaluate(test_set)

In [19]:
#batch size???

In [ ]:
def cnn

In [ ]:
def trainer(lr, model):
    name = "model_0524_(lr)" + str(lr) + ".h5"
    cp = keras.callbacks.ModelCheckpoint(name, save_best_only=True)
    es = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    tb = keras.callbacks.TensorBoard(run_logdir)
    history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[cp, es, tb])
    hisc = "history_(lr)" +  str(lr) + ".csv"
    hd = pd.DataFrame(history.history)
    with open(hisc, mode='w') as f:
        hd.to_csv(f)
    return history

In [ ]:
def grapher(lr):
    name = "history_(lr)" + str(lr) + ".csv"
    df = pd.read_csv(name, index_col=0)
    df.plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0, 1)
    plt.show()

In [ ]:
def test(lr):
    name = "model_0524_(lr)" + str(lr) + ".h5"
    model = keras.models.load_model(name)
    test = model.evaluate(X_test, y_test)
    return test

In [ ]:
model = 

In [ ]:
model2 = compiler(lr=0.01, model=model)

In [ ]:
model2.summary()

In [ ]:
trainer(0.01, model2)

In [ ]:
grapher(0.01)

In [ ]:
test(0.01)